# Inspect postproces for pe_present_on_image -> pe_exam

tested by exp010, fold0 validation

In [3]:

%cd ~/kaggle/rsna-str/workdir

/home/haito/kaggle/rsna-str/workdir


In [5]:


import pandas as pd
import pickle
import numpy as np
import matplotlib.pyplot as plt

from src.factory import *
from src.utils import *

In [6]:
DATADIR = Path("../input/rsna-str-pulmonary-embolism-detection/")

train = pd.read_csv(DATADIR / "train.csv")
pre = pd.read_csv(DATADIR / "split.csv")
train = train.merge(pre, on="StudyInstanceUID")

In [7]:
t = train[train.fold == 0]

In [8]:
studies = t.StudyInstanceUID.unique()


In [9]:
agg = t.groupby("StudyInstanceUID")["SOPInstanceUID"].apply(list)
agg_one = t.groupby("StudyInstanceUID").first()

In [10]:
def post1(p_arr, q=90):
    return np.percentile(p_arr, q=q)

In [11]:
from sklearn.metrics import log_loss

In [38]:
### exp 010
p = "output/010_pe_pos/out.valid.fold0"  # ep0
# p = "output/010_pe_pos/out.valid.fold0_ep1"

res = load_pickle(p)

raw_pred = pd.DataFrame({
    "sop": res["ids"],
    "pe_present_on_image": res["outputs"]["pe_present_on_image"],
    "rightsided_pe": res["outputs"]["rightsided_pe"],
    "leftsided_pe": res["outputs"]["leftsided_pe"],
    "central_pe": res["outputs"]["central_pe"],
})
raw_pred = raw_pred.set_index("sop")

In [111]:
# 001
_path = "output/001_base/fold0_valid.pickle"
res = load_pickle(_path)
raw_pred = pd.DataFrame({
    "sop": res["ids"],
    "pe_present": res["outputs"]
})
raw_pred = raw_pred.set_index("sop")

In [ ]:
""" exp 001 model (ep1)
q=90.00 logloss:0.4335881782152035
q=91.00 logloss:0.4261582133852183
q=92.00 logloss:0.41820680933855464
q=93.00 logloss:0.40965706920377787
q=94.00 logloss:0.40250102547323763
q=95.00 logloss:0.399617003382453   <=== たまたま Best
q=96.00 logloss:0.4014378415196566
q=97.00 logloss:0.40659142078878496
q=98.00 logloss:0.4293089602081556
q=99.00 logloss:0.4881203309768597
"""

In [ ]:
""" exp 010 model (ep0)
q=90 logloss:0.580963053992852
q=91 logloss:0.5561516892147172
q=92 logloss:0.5302883445027334
q=93 logloss:0.5057102176781858
q=94 logloss:0.48133613115138896
q=95 logloss:0.4547570238934727   <=========== Bad
q=96 logloss:0.4293633715604374
q=97 logloss:0.40440298977917516
q=98 logloss:0.383646086091538
q=98.80 logloss:0.37546814335089845
q=98.90 logloss:0.3744332339894111   <=========== Mininum for fold0
q=99.00 logloss:0.3745150839621268
q=99 logloss:0.374515083962127
q=99.10 logloss:0.3755236864711825
q=99.20 logloss:0.3776048971290255
q=99.5 logloss:0.389937762524077
q=99.7 logloss:0.40671684673171016
q=99.9 logloss:0.43696034136869655
q=99.95 logloss:0.44722869414163663
q=99.99 logloss:0.4569151069554547
"""

In [ ]:
""" exp 010 model (ep1)
q=90.00 logloss:0.5279595949554328
q=91.00 logloss:0.5052608242921037
q=92.00 logloss:0.4834110695747726
q=93.00 logloss:0.46385352130528656
q=94.00 logloss:0.4473139354706962
q=95.00 logloss:0.43102377556895444
q=96.00 logloss:0.41591824329556415
q=97.00 logloss:0.40408809195191403   <================ Best. epが進むほど最適なpercentilがさがる. epが進むとOver-confになるからだと思われる. exp001,best 0.3996
q=98.00 logloss:0.4081253578858078
q=99.00 logloss:0.4430865048629079
"""

In [59]:
MEANS_IND = 0.020484822355039723

#for q in [97,97.5,98, 98.5,99,99.5]:
#for q in np.arange(98.5, 99.3, 0.1):
#for q in np.arange(90.0, 99.9, 1):
for q in [98, 98.9]:

    LABELS = []
    PREDS = []
    LABELS_RIGHT = []
    PREDS_RIGHT = []
    PREDS_RIGHT2 = []
    LABELS_LEFT= []
    PREDS_LEFT = []
    PREDS_LEFT2 = []
    LABELS_CENT = []
    PREDS_CENT = []
    PREDS_CENT2 = []

    for study in t.StudyInstanceUID.unique():
        sops = agg.loc[study]
        label = agg_one.loc[study]
        label_is_pe = int((not label.indeterminate) and (not label.negative_exam_for_pe))
        LABELS.append(label_is_pe)

        LABELS_RIGHT.append(label.rightsided_pe)
        LABELS_LEFT.append(label.leftsided_pe)
        LABELS_CENT.append(label.central_pe)

        prediction =  raw_pred.loc[sops]  # preds for current study
        # pe pre
        probs_pe_present = prediction.pe_present_on_image
        pe_prob = post1(probs_pe_present, q=q)
        PREDS.append(pe_prob)

        ### rightsided
        ave_right = np.clip( np.sum( prediction.rightsided_pe ) / np.sum( prediction.pe_present_on_image ), 0, 1)
        ave_left  = np.clip( np.sum( prediction.leftsided_pe )  / np.sum( prediction.pe_present_on_image ), 0, 1)
        ave_cent  = np.clip( np.sum( prediction.central_pe )    / np.sum( prediction.pe_present_on_image ), 0, 1)

        if 0:
            PREDS_RIGHT.append( (1-MEANS_IND) * pe_prob *  ave_right)
            PREDS_LEFT.append ( (1-MEANS_IND) * pe_prob *  ave_left)
            PREDS_CENT.append ( (1-MEANS_IND) * pe_prob *  ave_cent)
        elif 0:
            PREDS_RIGHT.append( pe_prob *  ave_right)
            PREDS_LEFT.append ( pe_prob *  ave_left)
            PREDS_CENT.append ( pe_prob *  ave_cent)
        elif 0:
            PREDS_RIGHT.append( (1-MEANS_IND) * np.percentile( prediction.rightsided_pe, 99 ) )
            PREDS_LEFT .append( (1-MEANS_IND) * np.percentile( prediction.leftsided_pe, 99 ) )
            PREDS_CENT .append( (1-MEANS_IND) * np.percentile( prediction.central_pe, 99 ) )
        elif 1:
            PREDS_RIGHT.append( np.percentile( prediction.rightsided_pe, 99 ) * 1.15 )
            PREDS_LEFT .append( np.percentile( prediction.leftsided_pe, 99 )  * 1.15 )
            PREDS_CENT .append( np.percentile( prediction.central_pe, 99 )    * 1.15 )

        PREDS_RIGHT2.append( pe_prob * 0.849707702540123 + (1-pe_prob) * 0 )
        PREDS_LEFT2.append ( pe_prob * 0.6957545475146886 + (1-pe_prob) * 0 )
        PREDS_CENT2.append ( pe_prob * 0.1796915446534345 + (1-pe_prob) * 0 )

        # print(study, label_is_pe, pe_prob)

    print(f"q={q:.2f} logloss:{log_loss(LABELS, PREDS)}")

    print("RIGHT")
    print(f"    now  logloss:{log_loss(LABELS_RIGHT, PREDS_RIGHT)}")
    print(f"    past logloss:{log_loss(LABELS_RIGHT, PREDS_RIGHT2)}")
    print(f"    base logloss:{log_loss(LABELS_RIGHT, len(LABELS_RIGHT)*[np.mean(LABELS_RIGHT)] )}")

    print("LEFT")
    print(f"   logloss:{log_loss(LABELS_LEFT, PREDS_LEFT)}")
    print(f"   logloss:{log_loss(LABELS_LEFT, PREDS_LEFT2)}")
    print(f"   logloss:{log_loss(LABELS_LEFT, len(LABELS_LEFT)*[np.mean(LABELS_LEFT)] )}")

    print("CENTRAL")
    print(f"   logloss:{log_loss(LABELS_CENT, PREDS_CENT)}")
    print(f"   logloss:{log_loss(LABELS_CENT, PREDS_CENT2)}")
    print(f"   logloss:{log_loss(LABELS_CENT, len(LABELS_CENT)*[np.mean(LABELS_CENT)] )}")


q=98.00 logloss:0.383646086091538
RIGHT
    now  logloss:0.3451702421090064
    past logloss:0.3617282748788149
    base logloss:0.5762224411886869
RIGHT
   logloss:0.3214743942633246
   logloss:0.3394795618701709
   logloss:0.5300883358743675
CENTRAL
   logloss:0.1386441131671042
   logloss:0.15336295710202394
   logloss:0.23192769500092492
q=98.90 logloss:0.374433233989411
RIGHT
    now  logloss:0.3451702421090064
    past logloss:0.35746662291195486
    base logloss:0.5762224411886869
RIGHT
   logloss:0.3214743942633246
   logloss:0.3401054436957059
   logloss:0.5300883358743675
CENTRAL
   logloss:0.1386441131671042
   logloss:0.15980939159137056
   logloss:0.23192769500092492


In [82]:
log_loss(LABELS, PREDS)

0.4547570238934727

In [ ]:
MEANS_IND = 0.020484822355039723

#for q in [97,97.5,98, 98.5,99,99.5]:
#for q in np.arange(98.5, 99.3, 0.1):
#for q in np.arange(90.0, 99.9, 1):
for q in [98.9]:

    LABELS = []
    PREDS = []
    LABELS_RIGHT = []
    PREDS_RIGHT = []
    PREDS_RIGHT2 = []
    LABELS_LEFT= []
    PREDS_LEFT = []
    PREDS_LEFT2 = []
    LABELS_CENT = []
    PREDS_CENT = []
    PREDS_CENT2 = []

    for study in t.StudyInstanceUID.unique():
        sops = agg.loc[study]
        label = agg_one.loc[study]
        label_is_pe = int((not label.indeterminate) and (not label.negative_exam_for_pe))
        LABELS.append(label_is_pe)

        LABELS_RIGHT.append(label.rightsided_pe)
        LABELS_LEFT.append(label.leftsided_pe)
        LABELS_CENT.append(label.central_pe)

        prediction =  raw_pred.loc[sops]  # preds for current study
        # pe pre
        probs_pe_present = prediction.pe_present_on_image
        pe_prob = post1(probs_pe_present, q=q)
        PREDS.append(pe_prob)

        ### rightsided
        ave_right = np.clip( np.sum( prediction.rightsided_pe ) / np.sum( prediction.pe_present_on_image ), 0, 1)
        ave_left  = np.clip( np.sum( prediction.leftsided_pe )  / np.sum( prediction.pe_present_on_image ), 0, 1)
        ave_cent  = np.clip( np.sum( prediction.central_pe )    / np.sum( prediction.pe_present_on_image ), 0, 1)
        # print(ave_right, label_is_right)
        PREDS_RIGHT.append( (1-MEANS_IND) * pe_prob *  ave_right)
        PREDS_LEFT.append( (1-MEANS_IND) * pe_prob *  ave_left)
        PREDS_CENT.append( (1-MEANS_IND) * pe_prob *  ave_cent)

        PREDS_RIGHT2.append( pe_prob * 0.849707702540123 + (1-pe_prob) * 0 )
        PREDS_LEFT2.append( pe_prob * 0.6957545475146886 + (1-pe_prob) * 0 )
        PREDS_CENT2.append( pe_prob * 0.1796915446534345 + (1-pe_prob) * 0 )

        # print(study, label_is_pe, pe_prob)

    print(f"q={q:.2f} logloss:{log_loss(LABELS, PREDS)}")

    print(f"q={q:.2f} rifht now  logloss:{log_loss(LABELS_RIGHT, PREDS_RIGHT)}")
    print(f"q={q:.2f} right past logloss:{log_loss(LABELS_RIGHT, PREDS_RIGHT2)}")
    print(f"q={q:.2f} super base logloss:{log_loss(LABELS_RIGHT, len(LABELS_RIGHT)*[np.mean(LABELS_RIGHT)] )}")

    print("RIGHT")
    print(f"   logloss:{log_loss(LABELS_RIGHT, PREDS_RIGHT)}")
    print(f"   logloss:{log_loss(LABELS_RIGHT, PREDS_RIGHT2)}")
    print(f"   logloss:{log_loss(LABELS_RIGHT, len(LABELS_RIGHT)*[np.mean(LABELS_RIGHT)] )}")

    print(f"q={q:.2f} rifht now  logloss:{log_loss(LABELS_RIGHT, PREDS_RIGHT)}")
    print(f"q={q:.2f} right past logloss:{log_loss(LABELS_RIGHT, PREDS_RIGHT2)}")
    print(f"q={q:.2f} super base logloss:{log_loss(LABELS_RIGHT, len(LABELS_RIGHT)*[np.mean(LABELS_RIGHT)] )}")
